## This Notebook implements Gaussian Process interpolation on data from each filter.
### After GP, it implements wavelets transform + PCA feature reducing methods.
### Finally it joins with label DF generated at gen_labels.ipynb resulting on a final df with all features and target values (redshifts and supernova class type).

In [16]:
%load_ext autoreload
%autoreload 2
import os 
import sys
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
module_path = os.path.abspath(os.path.join('..','..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.processing.functions import get_raw_files, get_df_filter_data, gplc, wavelets, get_df_PCA 

In [17]:
pd.options.display.max_rows = 6
pd.options.display.max_columns = 40

## Getting raw data

In [8]:
%load_ext memory_profiler
%memit
%time
files = get_raw_files(module_path + '/data/raw')

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 91.89 MiB, increment: 0.01 MiB
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [13]:
%memit
%time
df_filter_data = get_df_filter_data(files)
%memit
%time

peak memory: 98.14 MiB, increment: 0.00 MiB
CPU times: user 0 ns, sys: 7 µs, total: 7 µs
Wall time: 11.2 µs
peak memory: 224.07 MiB, increment: 0.00 MiB
CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 7.87 µs


In [18]:
df_filter_data

,ID,desg,desi,desr,desz,Xaxis
0,SN125764,"[[1.9879999999975553, 1.773, 3.612], [25.03099...","[[0.00399999999353895, 6.387, 7.619], [8.03499...","[[0.0, 2.777, 5.379], [18.175999999999476, 5.1...","[[0.9879999999975553, 7.467, 5.125], [7.034999...","[0.0, 0.8282828282828283, 1.6565656565656566, ..."
1,SN513286,"[[1.9689999999973224, 4.901, 3.206], [15.96499...","[[10.944999999999709, 33.9, 30.96], [31.906000...","[[0.0, 7.563, 7.842], [1.9839999999967404, 0.9...","[[15.976999999998952, 4.139, 6.541], [29.94099...","[0.0, 1.604323232323246, 3.208646464646492, 4...."
2,SN501595,"[[1.9689999999973224, 4.233, 3.215], [10.08999...","[[0.01600000000325963, 7.958, 9.316], [2.0, 12...","[[0.0, 29.45, 7.868], [1.9839999999967404, 20....","[[0.9610000000029686, 20.99, 5.346], [9.940999...","[0.0, 1.271070707070737, 2.542141414141474, 3...."
...,...,...,...,...,...,...
21314,SN789717,"[[33.97299999999814, 2.381, 3.69], [41.9729999...","[[0.01599999999598367, 4.542, 3.146], [18.0860...","[[0.0, 2.838, 2.13], [4.082000000002154, 0.491...","[[0.03899999999703141, 1.41, 2.483], [3.067000...","[0.0, 1.5746969696969373, 3.1493939393938746, ..."
21315,SN147538,"[[0.0, 3.554, 2.637], [23.91399999999703, 1.51...","[[0.027000000001862645, 0.3069, 2.291], [7.932...","[[23.932999999997264, 0.6078, 0.956], [27.8319...","[[0.046999999998661224, 1.332, 2.384], [7.9569...","[0.0, 1.584242424242389, 3.168484848484778, 4...."
21316,SN52860,"[[0.0, 12.16, 10.48], [1.985000000000582, 0.89...","[[10.961000000002969, 50.2, 30.97], [22.0, 5.5...","[[15.989000000001397, 2.418, 5.4], [37.9420000...","[[0.9770000000062282, 1.835, 5.328], [9.957000...","[0.0, 1.745343434343478, 3.490686868686956, 5...."


In [19]:
df_filter_data.to_pickle(module_path + '/data/structured/df_filter_data.pkl')

## Gaussian Process
- nearly 20 hours to run

- mask filter cuts 28 objetcs (21319 - 21291)

In [ ]:
df = pd.read_pickle(module_path + '/data/structured/df_filter_data.pkl')
mask = (df.desg.str.len() > 1) & (df.desi.str.len() > 1) & (df.desr.str.len() > 1) & (df.desz.str.len() > 1) 
df = df.loc[mask]

df['desg_GP'] = df.apply(lambda df: gplc(df.ID, df.desg[:,0],df.desg[:,1],df.desg[:,2]), axis=1) 
#df.to_pickle("df_GP_scikit_desg.pkl")
df['desi_GP'] = df.apply(lambda df: gplc(df.ID, df.desi[:,0],df.desi[:,1],df.desi[:,2]), axis=1) 
#df.to_pickle("df_GP_scikit_desi.pkl")
df['desr_GP'] = df.apply(lambda df: gplc(df.ID, df.desr[:,0],df.desr[:,1],df.desr[:,2]), axis=1) 
#df.to_pickle("df_GP_scikit_desr.pkl")
df['desz_GP'] = df.apply(lambda df: gplc(df.ID, df.desz[:,0],df.desz[:,1],df.desz[:,2]), axis=1) 
#df.to_pickle("df_GP_scikit_desz.pkl")
df_GP = df

In [21]:
df_GP 

,ID,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP
0,SN554563,"[[0.0, 18.98, 10.49], [1.985000000000582, 0.89...","[[0.03200000000651926, 8.001, 9.313], [10.9610...","[[2.0, 5.875, 2.431], [10.121000000006461, 6.9...","[[15.993000000002212, 1.763, 6.543], [29.95700...","[0.0, 1.4326868686868988, 2.8653737373737975, ...","([0.0, 1.3517272727273086, 2.7034545454546173,...","([0.03200000000651926, 1.4642525252590228, 2.8...","([2.0, 3.412292929292957, 4.824585858585914, 6...","([15.993000000002212, 17.264141414143634, 18.5..."
1,SN454964,"[[0.0, 4.359, 6.2], [4.9799999999959255, 33.82...","[[0.010999999998603016, 3.629, 6.148], [4.9959...","[[0.004000000000814907, 0.8701, 3.584], [14.21...","[[5.004000000000815, 4.406, 7.695], [24.139999...","[0.0, 1.2527575757575864, 2.505515151515173, 3...","([0.0, 1.1215252525252062, 2.2430505050504124,...","([0.010999999998603016, 1.253707070705658, 2.4...","([0.004000000000814907, 1.24669696969773, 2.48...","([5.004000000000815, 6.206212121212938, 7.4084..."
2,SN537864,"[[0.0, 0.4987, 4.764], [11.969000000004598, 8....","[[4.105999999999767, 1.357, 3.683], [13.090000...","[[0.008000000001629815, 3.18, 2.13], [13.07899...","[[3.0750000000043656, 1.838, 3.729], [4.137000...","[0.0, 1.1715252525252502, 2.3430505050505004, ...","([0.0, 1.1417777777777784, 2.283555555555557, ...","([4.105999999999767, 5.236050505050272, 6.3661...","([0.008000000001629815, 1.1792828282844798, 2....","([3.0750000000043656, 4.214080808085176, 5.353..."
...,...,...,...,...,...,...,...,...,...,...
21314,SN51813,"[[14.859000000004016, 11.64, 11.89], [30.91799...","[[12.984000000004016, 5.321, 4.986], [22.88599...","[[0.0, 1.53, 1.444], [14.86699999999837, 2.602...","[[2.014999999999418, 3.934, 3.402], [13.014999...","[0.0, 0.7759595959595931, 1.5519191919191861, ...","([14.859000000004016, 15.47476767677165, 16.09...","([12.984000000004016, 13.628808080812053, 14.2...","([0.0, 0.7659393939394075, 1.531878787878815, ...","([2.014999999999418, 2.7200606060600188, 3.425..."
21315,SN599115,"[[0.0, 3.229, 7.693], [10.014999999999418, 1.1...","[[0.027000000001862645, 8.224, 5.11], [3.01099...","[[1.0, 0.9777, 3.85], [2.0429999999978463, 0.4...","[[1.0429999999978463, 2.444, 4.848], [2.085999...","[0.0, 1.2428181818181518, 2.4856363636363037, ...","([0.0, 0.9397070707070243, 1.8794141414140486,...","([0.027000000001862645, 1.269262626264456, 2.5...","([1.0, 2.2322828282828198, 3.46456565656564, 4...","([1.0429999999978463, 2.275282828280666, 3.507..."
21316,SN308423,"[[29.875, 4.829, 11.89], [53.88699999999517, 9...","[[0.0, 2.278, 4.54], [7.995999999999185, 3.879...","[[2.069999999999709, 0.06183, 1.853], [23.0239...","[[0.01999999999679858, 6.346, 3.061], [30.0159...","[0.0, 0.9276363636363202, 1.8552727272726404, ...","([29.875, 30.44022222222217, 31.00544444444434...","([0.0, 0.9276363636363202, 1.8552727272726404,...","([2.069999999999709, 2.9667070707067555, 3.863...","([0.01999999999679858, 0.8968888888856681, 1.7..."


In [ ]:
df_GP.to_pickle(module_path + '/data/structured/df_GP.pkl')

## Wavelets

In [28]:
df_GP = pd.read_pickle(module_path + '/data/structured/df_GP.pkl')
df_GP['wavelets'] = df_GP.apply(wavelets, axis = 1)
df_GP_wav = df_GP

In [29]:
df_GP_wav

,ID,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets
0,SN554563,"[[0.0, 18.98, 10.49], [1.985000000000582, 0.89...","[[0.03200000000651926, 8.001, 9.313], [10.9610...","[[2.0, 5.875, 2.431], [10.121000000006461, 6.9...","[[15.993000000002212, 1.763, 6.543], [29.95700...","[0.0, 1.4326868686868988, 2.8653737373737975, ...","([0.0, 1.3517272727273086, 2.7034545454546173,...","([0.03200000000651926, 1.4642525252590228, 2.8...","([2.0, 3.412292929292957, 4.824585858585914, 6...","([15.993000000002212, 17.264141414143634, 18.5...","[15.808392550321383, 18.638950202591968, 20.91..."
1,SN454964,"[[0.0, 4.359, 6.2], [4.9799999999959255, 33.82...","[[0.010999999998603016, 3.629, 6.148], [4.9959...","[[0.004000000000814907, 0.8701, 3.584], [14.21...","[[5.004000000000815, 4.406, 7.695], [24.139999...","[0.0, 1.2527575757575864, 2.505515151515173, 3...","([0.0, 1.1215252525252062, 2.2430505050504124,...","([0.010999999998603016, 1.253707070705658, 2.4...","([0.004000000000814907, 1.24669696969773, 2.48...","([5.004000000000815, 6.206212121212938, 7.4084...","[30.643224836287317, 35.67248978201266, 39.557..."
2,SN537864,"[[0.0, 0.4987, 4.764], [11.969000000004598, 8....","[[4.105999999999767, 1.357, 3.683], [13.090000...","[[0.008000000001629815, 3.18, 2.13], [13.07899...","[[3.0750000000043656, 1.838, 3.729], [4.137000...","[0.0, 1.1715252525252502, 2.3430505050505004, ...","([0.0, 1.1417777777777784, 2.283555555555557, ...","([4.105999999999767, 5.236050505050272, 6.3661...","([0.008000000001629815, 1.1792828282844798, 2....","([3.0750000000043656, 4.214080808085176, 5.353...","[10.627291552130997, 15.676212809195041, 19.95..."
...,...,...,...,...,...,...,...,...,...,...,...
21314,SN51813,"[[14.859000000004016, 11.64, 11.89], [30.91799...","[[12.984000000004016, 5.321, 4.986], [22.88599...","[[0.0, 1.53, 1.444], [14.86699999999837, 2.602...","[[2.014999999999418, 3.934, 3.402], [13.014999...","[0.0, 0.7759595959595931, 1.5519191919191861, ...","([14.859000000004016, 15.47476767677165, 16.09...","([12.984000000004016, 13.628808080812053, 14.2...","([0.0, 0.7659393939394075, 1.531878787878815, ...","([2.014999999999418, 2.7200606060600188, 3.425...","[13.954617071677685, 12.349012365000295, 10.98..."
21315,SN599115,"[[0.0, 3.229, 7.693], [10.014999999999418, 1.1...","[[0.027000000001862645, 8.224, 5.11], [3.01099...","[[1.0, 0.9777, 3.85], [2.0429999999978463, 0.4...","[[1.0429999999978463, 2.444, 4.848], [2.085999...","[0.0, 1.2428181818181518, 2.4856363636363037, ...","([0.0, 0.9397070707070243, 1.8794141414140486,...","([0.027000000001862645, 1.269262626264456, 2.5...","([1.0, 2.2322828282828198, 3.46456565656564, 4...","([1.0429999999978463, 2.275282828280666, 3.507...","[7.517572789571435, 6.52881398073386, 5.766703..."
21316,SN308423,"[[29.875, 4.829, 11.89], [53.88699999999517, 9...","[[0.0, 2.278, 4.54], [7.995999999999185, 3.879...","[[2.069999999999709, 0.06183, 1.853], [23.0239...","[[0.01999999999679858, 6.346, 3.061], [30.0159...","[0.0, 0.9276363636363202, 1.8552727272726404, ...","([29.875, 30.44022222222217, 31.00544444444434...","([0.0, 0.9276363636363202, 1.8552727272726404,...","([2.069999999999709, 2.9667070707067555, 3.863...","([0.01999999999679858, 0.8968888888856681, 1.7...","[13.035820614531577, 13.277986826716845, 13.49..."


In [30]:
df_GP_wav.to_pickle(module_path + '/data/structured/df_GP_wav.pkl')

## PCA

In [44]:
df_20PCA = get_df_PCA(df_GP_wav)

Explained variance ratio:  [7.75080477e-01 1.01425485e-01 4.61381626e-02 2.05184206e-02
 1.25061675e-02 9.63334368e-03 7.78383899e-03 6.93479216e-03
 3.43213957e-03 3.21499240e-03 2.15082714e-03 1.72901197e-03
 1.40127624e-03 1.14066078e-03 1.10067200e-03 9.20854977e-04
 7.70792006e-04 6.06373927e-04 5.74896303e-04 4.24922283e-04]

 Singular values:  [611412.03415977 221174.02953809 149173.17425849  99479.26030735
  77664.52751964  68163.06501515  61271.35792703  57833.21134671
  40685.81814524  39377.71932654  32207.96882184  28877.49939296
  25996.95377096  23455.18368534  23040.3749391   21074.44776892
  19280.98356476  17101.3676223   16651.57591156  14315.79057887]


In [45]:
df_20PCA

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20
0,-152.456150,116.958067,77.435307,13.471403,-51.955130,-18.168116,64.503646,28.600517,-27.774363,-8.120098,-20.397438,20.430515,-69.650398,75.310421,117.447984,-41.083396,-49.527872,19.079276,17.427959,1.112086
1,-312.552895,6.881269,-117.104000,115.002821,9.379711,-139.815067,23.009522,-95.772257,-71.053158,-89.069248,-6.423624,-40.945593,38.165859,-17.001822,7.106318,-35.068364,-10.663222,2.622628,-12.197105,-2.859290
2,-153.247121,140.551473,35.816624,-1.885185,-26.144828,70.170131,-56.272975,-69.939925,72.466017,-20.776364,39.708823,-1.412278,17.577747,3.413035,14.138187,-37.382939,24.262484,-31.411998,-7.438044,18.769166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21288,476.795587,-1067.956742,-360.601472,-232.028136,-174.974045,-144.276123,-483.082741,102.094460,126.465358,-183.045258,-47.553118,-106.139240,-126.098258,71.861387,109.377996,-149.822953,168.444143,182.911709,-49.802376,-94.982308
21289,-612.440524,126.730122,-66.678784,-27.158989,27.419128,-56.102780,8.534456,-1.585473,3.247079,-47.549219,-12.443284,-11.492505,-14.476083,-9.730008,20.852124,-41.093223,1.236326,26.710125,16.355957,-23.481584
21290,-643.285190,111.940726,-111.379727,18.716777,50.676215,-82.445315,0.816497,-95.388172,-36.080611,-55.134534,19.161812,-3.816460,33.418663,-18.363777,-30.355170,14.762046,20.998781,-16.922467,-37.872510,28.001539


In [46]:
df_GP_wav_PCA20 = pd.concat([df_GP_wav.reset_index().drop(columns = ['index']), df_20PCA], axis = 1, sort = True)

In [47]:
df_GP_wav_PCA20

,ID,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20
0,SN554563,"[[0.0, 18.98, 10.49], [1.985000000000582, 0.89...","[[0.03200000000651926, 8.001, 9.313], [10.9610...","[[2.0, 5.875, 2.431], [10.121000000006461, 6.9...","[[15.993000000002212, 1.763, 6.543], [29.95700...","[0.0, 1.4326868686868988, 2.8653737373737975, ...","([0.0, 1.3517272727273086, 2.7034545454546173,...","([0.03200000000651926, 1.4642525252590228, 2.8...","([2.0, 3.412292929292957, 4.824585858585914, 6...","([15.993000000002212, 17.264141414143634, 18.5...","[15.808392550321383, 18.638950202591968, 20.91...",-152.456150,116.958067,77.435307,13.471403,-51.955130,-18.168116,64.503646,28.600517,-27.774363,-8.120098,-20.397438,20.430515,-69.650398,75.310421,117.447984,-41.083396,-49.527872,19.079276,17.427959,1.112086
1,SN454964,"[[0.0, 4.359, 6.2], [4.9799999999959255, 33.82...","[[0.010999999998603016, 3.629, 6.148], [4.9959...","[[0.004000000000814907, 0.8701, 3.584], [14.21...","[[5.004000000000815, 4.406, 7.695], [24.139999...","[0.0, 1.2527575757575864, 2.505515151515173, 3...","([0.0, 1.1215252525252062, 2.2430505050504124,...","([0.010999999998603016, 1.253707070705658, 2.4...","([0.004000000000814907, 1.24669696969773, 2.48...","([5.004000000000815, 6.206212121212938, 7.4084...","[30.643224836287317, 35.67248978201266, 39.557...",-312.552895,6.881269,-117.104000,115.002821,9.379711,-139.815067,23.009522,-95.772257,-71.053158,-89.069248,-6.423624,-40.945593,38.165859,-17.001822,7.106318,-35.068364,-10.663222,2.622628,-12.197105,-2.859290
2,SN537864,"[[0.0, 0.4987, 4.764], [11.969000000004598, 8....","[[4.105999999999767, 1.357, 3.683], [13.090000...","[[0.008000000001629815, 3.18, 2.13], [13.07899...","[[3.0750000000043656, 1.838, 3.729], [4.137000...","[0.0, 1.1715252525252502, 2.3430505050505004, ...","([0.0, 1.1417777777777784, 2.283555555555557, ...","([4.105999999999767, 5.236050505050272, 6.3661...","([0.008000000001629815, 1.1792828282844798, 2....","([3.0750000000043656, 4.214080808085176, 5.353...","[10.627291552130997, 15.676212809195041, 19.95...",-153.247121,140.551473,35.816624,-1.885185,-26.144828,70.170131,-56.272975,-69.939925,72.466017,-20.776364,39.708823,-1.412278,17.577747,3.413035,14.138187,-37.382939,24.262484,-31.411998,-7.438044,18.769166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21288,SN51813,"[[14.859000000004016, 11.64, 11.89], [30.91799...","[[12.984000000004016, 5.321, 4.986], [22.88599...","[[0.0, 1.53, 1.444], [14.86699999999837, 2.602...","[[2.014999999999418, 3.934, 3.402], [13.014999...","[0.0, 0.7759595959595931, 1.5519191919191861, ...","([14.859000000004016, 15.47476767677165, 16.09...","([12.984000000004016, 13.628808080812053, 14.2...","([0.0, 0.7659393939394075, 1.531878787878815, ...","([2.014999999999418, 2.7200606060600188, 3.425...","[13.954617071677685, 12.349012365000295, 10.98...",476.795587,-1067.956742,-360.601472,-232.028136,-174.974045,-144.276123,-483.082741,102.094460,126.465358,-183.045258,-47.553118,-106.139240,-126.098258,71.861387,109.377996,-149.822953,168.444143,182.911709,-49.802376,-94.982308
21289,SN599115,"[[0.0, 3.229, 7.693], [10.014999999999418, 1.1...","[[0.027000000001862645, 8.224, 5.11], [3.01099...","[[1.0, 0.9777, 3.85], [2.0429999999978463, 0.4...","[[1.0429999999978463, 2.444, 4.848], [2.085999...","[0.0, 1.2428181818181518, 2.4856363636363037, ...","([0.0, 0.9397070707070243, 1.8794141414140486,...","([0.027000000001862645, 1.269262626264456, 2.5...","([1.0, 2.2322828282828198, 3.46456565656564, 4...","([1.0429999999978463, 2.275282828280666, 3.507...","[7.517572789571435, 6.52881398073386, 5.766703...",-612.440524,126.730122,-66.678784,-27.158989,27.419128,-56.102780,8.534456,-1.585473,3.247079,-47.549219,-12.443284,-11.492505,-14.476083,-9.730008,20.852124,-41.093223,1.236326,26.710125,16.355957,-23.481584


In [48]:
df_GP_wav_PCA20.to_pickle(module_path + '/data/structured/df_GP_wav_PCA20.pkl')

## Concat with redshifts and types

In [49]:
df_redshift = pd.read_pickle(module_path + '/data/structured/df_redshift.pkl')
df_redshift

,type,type_bool,REDSHIFT_SPEC
ID,,,
SN10001,II,False,0.59162
SN100023,II,False,0.79264
SN10005,Ia,True,1.08327
...,...,...,...
SN99921,II,False,0.69650
SN99946,Ia,True,0.40625
SN9999,II,False,0.54982


In [50]:
df = pd.concat([df_GP_wav_PCA20.set_index('ID'), df_redshift], axis = 1 , join = "inner", sort = 'True')

In [51]:
df

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SN10001,"[[0.0, 2.326, 4.792], [2.0, 4.617, 2.249], [9....","[[0.031000000002677552, 8.672, 5.439], [2.1560...","[[0.01600000000325963, 8.834, 5.339], [2.13999...","[[0.06199999999807915, 6.026, 4.693], [2.18699...","[0.0, 1.1313535353535435, 2.262707070707087, 3...","([0.0, 0.818888888888886, 1.637777777777772, 2...","([0.031000000002677552, 1.141676767679467, 2.2...","([0.01600000000325963, 1.076565656568908, 2.13...","([0.06199999999807915, 1.1927272727253795, 2.3...","[9.251771363349247, 8.972870155014132, 8.79160...",-627.606441,282.667898,-58.310795,25.674751,60.694436,-64.764610,105.831517,-20.433773,-16.436875,-11.941724,-23.881895,29.151572,12.431821,6.048710,-11.165995,23.954122,18.206974,6.125705,9.448973,-0.077951,II,False,0.59162
SN100023,"[[6.984000000004016, 5.679, 6.387], [7.9920000...","[[7.00800000000163, 4.698, 7.145], [8.02300000...","[[0.0, 16.04, 17.59], [6.992000000005646, 19.8...","[[4.121000000006461, 18.01, 3.601], [7.0230000...","[0.0, 1.3437474747475018, 2.6874949494950036, ...","([6.984000000004016, 8.256929292933277, 9.5298...","([7.00800000000163, 8.189737373738987, 9.37147...","([0.0, 0.8994242424242942, 1.7988484848485884,...","([4.121000000006461, 5.423121212127635, 6.7252...","[12.22728787505526, 11.357060718395505, 10.716...",-504.040778,312.171082,-78.632844,41.877099,42.770978,-58.413274,114.501518,-1.783929,33.056277,17.795268,-55.479337,46.750660,41.486147,33.710197,-11.581964,28.655932,19.743617,4.619561,37.266783,-16.595923,II,False,0.79264
SN10005,"[[10.027000000001863, 4.996, 8.081], [33.94500...","[[0.015000000006693881, 0.2977, 2.206], [18.94...","[[0.0, 1.033, 1.325], [13.015000000006694, 0.4...","[[0.03500000000349246, 1.826, 3.029], [48.875,...","[0.0, 1.0381919191919462, 2.0763838383838924, ...","([10.027000000001863, 10.782565656567513, 11.5...","([0.015000000006693881, 1.0530404040470573, 2....","([0.0, 1.0281717171717606, 2.056343434343521, ...","([0.03500000000349246, 1.0222929292964054, 2.0...","[2.000004411755352, 2.000003547336676, 2.00000...",-750.982205,121.777397,-119.276362,-41.573979,3.954831,-64.398708,-71.890124,-138.418639,35.188458,2.706010,-42.582112,-14.480661,36.831220,-26.554909,15.768490,16.436792,17.114796,-20.292959,-24.222562,-11.295566,Ia,True,1.08327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN99921,"[[0.0, 8.327, 4.797], [2.0, 9.834, 2.261], [9....","[[0.031000000002677552, 22.78, 5.454], [2.1560...","[[0.01600000000325963, 26.73, 5.361], [2.13999...","[[0.06199999999807915, 15.19, 4.7], [2.1869999...","[0.0, 1.2923333333333222, 2.5846666666666445, ...","([0.0, 1.1299292929293416, 2.2598585858586833,...","([0.031000000002677552, 1.3230202020228414, 2....","([0.01600000000325963, 1.3080606060638358, 2.6...","([0.06199999999807915, 1.2726666666647435, 2.4...","[14.025137598441496, 16.23859348218353, 17.966...",-387.257314,432.672718,-56.655346,-20.165072,96.749213,-27.368577,135.903932,23.258086,17.132275,9.157236,-5.636551,75.910066,34.345964,30.582545,-38.454611,33.695746,32.105618,-3.304426,32.695384,-16.237329,II,False,0.69650
SN99946,"[[11.961000000002969, 1.526, 5.683], [13.05500...","[[0.01599999999598367, 4.95, 3.147], [3.034999...","[[0.0, 0.4826, 2.129], [2.995999999999185, 0.7...","[[0.03899999999703141, 1.045, 2.483], [12.0389...","[0.0, 1.3626161616161374, 2.725232323232275, 4...","([11.961000000002969, 13.202242424245336, 14.4...","([0.01599999999598367, 1.3782121212081455, 2.7...","([0.0, 1.362181818181816, 2.724363636363632, 4...","([0.03899999999703141, 1.4012222222192594, 2.7...","[3.7693816011787566, 5.06053615747151, 6.78000...",1392.358593,125.615871,1214.437894,-79.047649,-194.479029,793.095693,-167.556662,-122.945883,-454.043416,325.10011

In [52]:
df.to_pickle(module_path + '/data/structured/df.pkl')